# From $I(Q)$ to $G(\delta)$ using a numerical Hankel transform
Straight numpy implementation for a sphere form factor of the following paper:

Bakker, J. H., Washington, A. L., Parnell, S. R., Van Well, A. A., Pappas, C., & Bouwman, W. G. (2020). Analysis of SESANS data by numerical Hankel transform implementation in SasView. Journal of Neutron Research, 22(1), 57-70. https://doi.org/10.3233/jnr-200154

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import j0

In [ ]:
def form_factor(Q, R):
    return (3 * (np.sin(Q * R) - Q * R * np.cos(Q * R))/ (Q * R) ** 3)**2

In [ ]:
R =50e-9
Q_min = 0.01 / R
Q_max = 10 / R
Q = np.linspace(Q_min, Q_max, 1000)
ff = form_factor(Q,R)
plt.plot(Q * 1e-10, ff)

In [ ]:
def setup_Q_delta_arrays(delta_min, delta_max, Z):
  delta = np.linspace(delta_min, delta_max, Z)
  Q_max = 2 * np.pi / delta_min
  dQ = 0.1 * 2 * np.pi / (Z * (delta_max - delta_min))
  N = int(np.ceil(Q_max / dQ))
  Qn = np.arange(1,N+1) * dQ
  return delta, Qn, dQ, N

delta_min = R / 10
delta_max = 3 * R
Z = 250
delta, Qn, dQ, N =setup_Q_delta_arrays(delta_min, delta_max, Z)
dQ, Qn, N

In [ ]:
def create_hankel_kernels(Qn, dQ, delta, Z):
  j0_Q_delta = j0(np.outer(Qn,delta))
  Q_mat = np.tile(Qn, (Z,1)).transpose()
  H_kernel = Q_mat * j0_Q_delta * dQ
  H_0 = Qn*dQ
  return H_0, H_kernel
H_0, H_kernel = create_hankel_kernels(Qn, dQ, delta, Z)

In [ ]:
def compute_G_matrices(I_Q, H_0, H_kernel, Z):
  G_full =  np.tile(I_Q, (Z,1)).transpose() * H_kernel
  G_full_0 = I_Q * H_0
  return G_full, G_full_0
I_Q = form_factor(Qn,R) * 0.5 * R ** 2
G_full, G_full_0 = compute_G_matrices(I_Q, H_0, H_kernel, Z)
G_delta_num = np.sum(G_full,axis=0)
sigma_t_num = np.sum(G_full_0,axis=0)
G_delta_num.shape, sigma_t_num

In [ ]:
def G_0(xi):
    res = np.zeros_like(xi)
    res[xi>=2.0] = 0
    valid_xi = xi[xi<2.0]
    res[xi<2.0] = np.sqrt(1 - (valid_xi / 2) ** 2) * (1 + valid_xi ** 2 / 8)\
         + 1 / 2 * valid_xi ** 2 * (1 - (valid_xi / 4 ) ** 2) * np.log(valid_xi / (2 + np.sqrt(4 - valid_xi ** 2)))
    return res
G_0_num = G_delta_num / sigma_t_num

xi = delta / R
G_0_an = G_0(xi)
plt.plot(delta * 1e9,G_0_an)
plt.plot(delta * 1e9,G_0_num)
plt.xlabel(r'$\delta$ [nm]')

# Limited $Q$-range effects

In [ ]:
sigma_partials = np.cumsum(G_full_0) / sigma_t_num
plt.plot(Qn * 1e-10, sigma_partials)
plt.xlabel(r'$Q$ [$\AA^{-1}$]')

In [ ]:
G_partials = np.cumsum(G_full, axis = 0) / sigma_t_num
G_partials.shape

In [ ]:
def indices_within_range(x, a, b):
    return np.where((x >= a) & (x <= b))[0]
Q_max_plot = 2 * np.pi / R
Q_indices = indices_within_range(Qn,0, Q_max_plot)
Q_max_plot * 1e-10

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(Qn[Q_indices],delta, indexing='ij')
surf = ax.plot_surface(X * 1e-10, Y  * 1e9, G_partials[Q_indices,:], cmap='viridis', edgecolor='none')
cbar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, pad=0.12)
ax.set_xlabel(r'$Q$ [$\AA^{-1}$]')
ax.set_ylabel(r'$\delta$ [nm]')
ax.set_zlabel(r'$G_{exp}(\delta)$')
ax.view_init(elev=30, azim=130)  # Adjust these values as needed
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(Qn[Q_indices],delta, indexing='ij')

# Supposed to represent
G_0_partial = G_partials[Q_indices,:] / np.tile(sigma_partials[Q_indices], (Z,1)).transpose()
P_partial = np.exp(np.tile(sigma_partials[Q_indices], (Z,1)).transpose() * (G_0_partial - 1))

surf = ax.plot_surface(X * 1e-10, Y  * 1e9, G_0_partial, cmap='viridis', edgecolor='none')
cbar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, pad=0.12)
ax.set_xlabel(r'$Q$ [$\AA^{-1}$]')
ax.set_ylabel(r'$\delta$ [nm]')
ax.set_zlabel(r'$G_{0,exp}(\delta)$')
plt.title(r"$G_{0,exp}(\delta)$ as a function of the integrated Q-range")
ax.view_init(elev=30, azim=130)  # Adjust these values as needed
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(Qn[Q_indices],delta, indexing='ij')

surf = ax.plot_surface(X * 1e-10, Y  * 1e9, P_partial, cmap='viridis', edgecolor='none')
cbar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, pad=0.1)
ax.set_xlabel(r'$Q$ [$\AA^{-1}$]')
ax.set_ylabel(r'$\delta$ [nm]')
plt.title(r"$P_{exp}(\delta)$ as a function of the integrated Q-range")
ax.set_zlabel(r'$P_{exp}(\delta)$')
ax.view_init(elev=30, azim=130)  # Adjust these values as needed
plt.show()

In [ ]:
def get_Q_ix(Q):
  return int(Q/dQ)

Q_lims = [0.001e10, 0.002e10, 0.003e10, 0.004e10, 0.005e10, 0.007e10, 0.012e10]
for Q_lim in Q_lims:
  plt.plot(delta * 1e9, G_0_partial[get_Q_ix(Q_lim),:], label=r'$Q_{max}$ = ' + str(Q_lim * 1e-10) + r'$\AA^{-1}$')
  plt.xlabel(r'$\delta$ [nm]')
  plt.ylabel(r'$G_{0,exp}(\delta)$')
plt.legend()
plt.grid()

In [ ]:
for Q_lim in Q_lims:
  plt.plot(delta * 1e9, P_partial[get_Q_ix(Q_lim),:], label=r'$Q_{max}$ = ' + str(Q_lim * 1e-10) + r'$\AA^{-1}$')
  plt.xlabel(r'$\delta$ [nm]')
  plt.ylabel(r'$G_{exp}(\delta)$')
plt.legend()
plt.grid()
plt.xscale('log')